<a href="https://colab.research.google.com/github/1NetoDev/HabituAI/blob/main/HabituAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Célula 1: Instalações e Configuração da API Key

# Instalações de bibliotecas Python
!pip install -q gradio google-generativeai markdown2 Pillow PyPDF2 pytesseract
# Adicionado markdown2 aqui ^ (Pillow, PyPDF2, Pytesseract são opcionais se não usar upload de arquivos)

# ... (resto da Célula 1 como antes, com a configuração da API Key) ...
print("Instalações de libs Python concluídas.") # Certifique-se que markdown2 foi listado

# Configuração da API Key do Google AI
import os
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY_GLOBAL = None
try:
    GOOGLE_API_KEY_GLOBAL = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY_GLOBAL is None:
        raise ValueError("Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    genai.configure(api_key=GOOGLE_API_KEY_GLOBAL)
    print("Chave da API do Google AI configurada com sucesso para HabituAI!")
except Exception as e:
    print(f"ATENÇÃO: Erro ao configurar a chave da API do Google AI: {e}")

Instalações de libs Python concluídas.
Chave da API do Google AI configurada com sucesso para HabituAI!


In [40]:
# Célula 2: Definições de Backend do HabituAI
# ... (início da célula, variáveis globais, função chamar_ia_para_habitos) ...

# INSTRUCAO_SISTEMA_BOAS_VINDAS (permanece igual)
INSTRUCAO_SISTEMA_BOAS_VINDAS = """
Você é 'HabituAI', seu Guia Pessoal para a Jornada de Hábitos.
Sua primeira tarefa é dar as boas-vindas ao 'Aventureiro' (usuário) de forma amigável, temática (RPG/jornada) e concisa.
Faça UMA pergunta clara e direta para que ele comece a compartilhar o que deseja trabalhar.
Exemplo de pergunta: "Saudações, nobre Aventureiro! Sou HabituAI, e juntos vamos forjar hábitos lendários. Para começar nossa jornada, diga-me: qual grande meta você busca ou qual desafio de hábito quer conquistar primeiro?"
Use Markdown para formatação (ex: **negrito**). Mantenha a resposta inicial curta (2-3 frases mais a pergunta).
"""

INSTRUCAO_SISTEMA_ANALISE_E_PLANOS = """
Você é 'HabituAI', Guia Pessoal para a Jornada de Hábitos. O Aventureiro compartilhou informações ou está pedindo um novo plano.
Sua tarefa agora é:
1.  Analisar TODO o histórico da conversa para entender o contexto atual e os objetivos do Aventureiro.
2.  Se o Aventureiro estiver pedindo um novo conjunto de missões ou se as informações anteriores forem suficientes para um *novo* plano (ou um plano atualizado), SUGIRA de 2 a 4 'Missões' (hábitos).
    *   Se já existia um plano, você pode sugerir missões completamente novas OU variações/progressões das anteriores, conforme o contexto da conversa.
    *   Para cada Missão/Hábito:
        - Dê um nome temático (Ex: '**Missão: Despertar do Fênix**').
        - Defina uma META CLARA e ALCANÇAVEL (Ex: 'Acordar 15 minutos mais cedo para alongamento').
        - Atribua XP (entre 10 e 50) e GOLD (entre 1 e 5).
        - Especifique a FREQUÊNCIA (Ex: 'Diário', 'Seg, Qua, Sex').
3.  Se as informações ainda forem muito vagas para criar missões concretas, faça MAIS UMA ou DUAS perguntas MUITO ESPECÍFICAS para clarificar.
4.  SEMPRE que você apresentar um conjunto de missões (sejam iniciais ou um novo conjunto), APRESENTE este plano com o marcador EXATO "### 📜 Seu Novo Pergaminho de Missões 📜 ###" no início da seção do plano. (Mudei o nome para diferenciar do "Primeiro")
    - Cada missão deve começar com `**Missão: [Nome da Missão]**`.
    - Os detalhes devem ser itens de lista: `- Meta: [descrição]`, `- Frequência: [descrição]`, `- Recompensa: [XP] XP, [GOLD] Gold`.
5.  Após apresentar o plano (ou fazer mais perguntas), finalize sua resposta PERGUNTANDO ao Aventureiro se ele está pronto para estas novas missões, se deseja ajustes, ou respondendo à sua última pergunta.
Use Markdown para toda a formatação.
"""

# ... (função chamar_ia_para_habitos como antes) ...

# ... (Funções de Gamificação: resetar_jogo_e_conversa, get_estado_jogo_atualizado) ...

def tentar_parsear_plano_de_habitos_da_ia(texto_resposta_ia_str):
    global plano_de_habitos_gerado, id_habito_counter

    # Procurar pelo novo marcador ou pelo antigo para retrocompatibilidade, se necessário
    marcador_plano = "### 📜 Seu Novo Pergaminho de Missões 📜 ###"
    if marcador_plano not in texto_resposta_ia_str:
        # Tenta o marcador antigo se o novo não for encontrado
        marcador_plano_antigo = "### 📜 Seu Primeiro Pergaminho de Missões 📜 ###"
        if marcador_plano_antigo not in texto_resposta_ia_str:
            print(f"DEBUG Backend - tentar_parsear: Marcador '{marcador_plano}' (ou antigo) não encontrado.")
            return False # Não é um novo plano de missões
        else:
            marcador_plano = marcador_plano_antigo # Usa o antigo se encontrado

    # Limpa o plano anterior ANTES de adicionar novos itens
    plano_de_habitos_gerado = []
    id_habito_counter = 0
    habitos_extraidos_temp = []

    linhas = texto_resposta_ia_str.split(marcador_plano, 1)[-1].split('\n') # Pega só o que vem depois do marcador
    missao_atual_nome_temp = None
    detalhes_habito_atual_temp = {}

    for linha_str in linhas:
        linha_strip = linha_str.strip()

        if linha_strip.startswith("**Missão:"):
            if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
                id_habito_counter += 1
                detalhes_habito_atual_temp['id'] = id_habito_counter
                detalhes_habito_atual_temp['concluido'] = False
                detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada')
                detalhes_habito_atual_temp.setdefault('xp', 20)
                detalhes_habito_atual_temp.setdefault('gold', 2)
                habitos_extraidos_temp.append(detalhes_habito_atual_temp)

            missao_atual_nome_temp = linha_strip.replace("**Missão:", "").replace("**", "").strip()
            detalhes_habito_atual_temp = {"nome": missao_atual_nome_temp}

        elif missao_atual_nome_temp and linha_strip.startswith("- Meta:"):
            detalhes_habito_atual_temp['meta'] = linha_strip.replace("- Meta:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Frequência:"):
            detalhes_habito_atual_temp['frequencia'] = linha_strip.replace("- Frequência:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Recompensa:"):
            recompensa_str = linha_strip.replace("- Recompensa:", "").strip().lower()
            try:
                xp_val, gold_val = 20, 2 # Defaults
                partes_recompensa = recompensa_str.split(',')
                for parte in partes_recompensa:
                    parte_strip = parte.strip()
                    if "xp" in parte_strip: xp_val = int(parte_strip.replace("xp", "").strip())
                    elif "gold" in parte_strip: gold_val = int(parte_strip.replace("gold", "").strip())
                detalhes_habito_atual_temp['xp'] = xp_val
                detalhes_habito_atual_temp['gold'] = gold_val
            except ValueError:
                print(f"DEBUG Backend - tentar_parsear: Erro recompensa '{recompensa_str}'. Defaults.")
                detalhes_habito_atual_temp['xp'], detalhes_habito_atual_temp['gold'] = 20, 2
        elif linha_strip == "" and missao_atual_nome_temp: # Linha vazia pode indicar fim de uma missão
            pass # A missão será salva quando a próxima "**Missão:" ou o fim do loop for alcançado
        elif not linha_strip.startswith(("- Meta:", "- Frequência:", "- Recompensa:", "**Missão:")) and missao_atual_nome_temp:
            # Se encontrar texto que não é parte da estrutura da missão, assume que a lista de missões acabou
            if 'meta' in detalhes_habito_atual_temp: # Salva a missão atual se estiver completa
                 id_habito_counter += 1
                 detalhes_habito_atual_temp['id'] = id_habito_counter
                 detalhes_habito_atual_temp['concluido'] = False
                 detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada'); detalhes_habito_atual_temp.setdefault('xp', 20); detalhes_habito_atual_temp.setdefault('gold', 2)
                 habitos_extraidos_temp.append(detalhes_habito_atual_temp)
            missao_atual_nome_temp = None # Reseta para não continuar adicionando a uma missão antiga
            detalhes_habito_atual_temp = {}
            break # Para de processar linhas após o fim percebido das missões


    # Adiciona a última missão parseada se houver
    if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
        id_habito_counter += 1
        detalhes_habito_atual_temp['id'] = id_habito_counter
        detalhes_habito_atual_temp['concluido'] = False
        detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada'); detalhes_habito_atual_temp.setdefault('xp', 20); detalhes_habito_atual_temp.setdefault('gold', 2)
        habitos_extraidos_temp.append(detalhes_habito_atual_temp)

    if habitos_extraidos_temp:
        plano_de_habitos_gerado = habitos_extraidos_temp
        print(f"DEBUG Backend: NOVO Plano de hábitos ATUALIZADO da IA: {plano_de_habitos_gerado}")
        return True
    else:
        print("DEBUG Backend: Nenhum hábito concreto foi parseado da resposta da IA (após o marcador de pergaminho).")
        return False

# ... (função marcar_habitos_concluidos_ui como antes) ...
def marcar_habitos_concluidos_ui(lista_nomes_habitos_concluidos_da_ui):
    global xp_total, gold_total, plano_de_habitos_gerado, nivel_usuario
    print(f"DEBUG Backend: marcar_habitos_concluidos_ui - Recebido da UI: {lista_nomes_habitos_concluidos_da_ui}")
    for habito_dict in plano_de_habitos_gerado:
        nome_habito_no_plano = habito_dict.get("nome")
        estava_concluido = habito_dict.get("concluido", False)
        esta_na_lista_ui = nome_habito_no_plano in lista_nomes_habitos_concluidos_da_ui
        if esta_na_lista_ui and not estava_concluido:
            habito_dict["concluido"] = True; xp_total += habito_dict.get("xp", 0); gold_total += habito_dict.get("gold", 0)
            print(f"DEBUG Backend: Hábito '{nome_habito_no_plano}' MARCADO. +{habito_dict.get('xp',0)}XP, +{habito_dict.get('gold',0)}Gold")
        elif not esta_na_lista_ui and estava_concluido:
            habito_dict["concluido"] = False; xp_total -= habito_dict.get("xp", 0); gold_total -= habito_dict.get("gold", 0)
            if xp_total < 0: xp_total = 0;
            if gold_total < 0: gold_total = 0
            print(f"DEBUG Backend: Hábito '{nome_habito_no_plano}' DESMARCADO. Pontos revertidos.")
    xp_necessario_para_level_up = nivel_usuario * 100
    if xp_total >= xp_necessario_para_level_up and xp_necessario_para_level_up > 0 :
        niveis_ganhos = xp_total // xp_necessario_para_level_up
        nivel_usuario += niveis_ganhos; xp_total = xp_total % xp_necessario_para_level_up
        print(f"DEBUG Backend: LEVEL UP! Você alcançou o Nível {nivel_usuario}! XP atual: {xp_total}")
    return get_estado_jogo_atualizado()

print("Célula 2: Funções de Backend do HabituAI (Completas e Corrigidas para Novos Planos) DEFINIDAS.")

Célula 2: Funções de Backend do HabituAI (Completas e Corrigidas para Novos Planos) DEFINIDAS.


In [21]:
# Célula 2.1: Estado da Sessão e Constantes do Sistema

# ... (outras variáveis de estado como antes) ...
id_habito_counter = 0 # Certifique-se que está aqui

# --- Instruções de Sistema para a IA ---
INSTRUCAO_SISTEMA_BOAS_VINDAS = """
Você é 'HabituAI', seu Guia Pessoal para a Jornada de Hábitos.
Sua primeira tarefa é dar as boas-vindas ao 'Aventureiro' (usuário) de forma amigável e concisa.
Faça UMA pergunta clara e direta para que ele comece a compartilhar o que deseja trabalhar.
Exemplo: "Olá, Aventureiro! Sou HabituAI, seu guia nesta jornada. Para começarmos, me diga: qual grande hábito você sonha em construir ou qual desafio de hábito gostaria de superar primeiro?"
Mantenha o tom encorajador e temático.
"""

INSTRUCAO_SISTEMA_ANALISE_E_PLANOS = """
Você é 'HabituAI'. O Aventureiro compartilhou seu(s) objetivo(s) inicial(is) ou respondeu suas perguntas.
Sua tarefa agora é:
1.  Analisar TODAS as respostas anteriores dele no histórico da conversa.
2.  Se as informações ainda forem muito vagas para criar hábitos concretos, faça MAIS UMA ou DUAS perguntas MUITO ESPECÍFICAS para clarificar. Mantenha o tom temático.
3.  Se as informações forem suficientes, SUGIRA de 2 a 4 'Missões Iniciais' (hábitos).
    Para cada Missão/Hábito:
    - Dê um nome temático (Ex: 'Elixir da Manhã', 'Trilha do Conhecimento').
    - Defina uma META CLARA e ALCANÇAVEL (Ex: 'Beber 500ml de água ao acordar', 'Ler 10 páginas de um livro').
    - Atribua XP (entre 10 e 50) e GOLD (entre 1 e 5).
    - Especifique a FREQUÊNCIA (Ex: 'Diário', '3x por semana').
4.  APRESENTE este plano como um pergaminho. Use formatação Markdown:
    - Inicie com "### 📜 Seu Primeiro Pergaminho de Missões 📜 ###".
    - Cada missão: `**Missão: [Nome]**`, `- Meta: ...`, `- Frequência: ...`, `- Recompensa: ... XP, ... Gold`.
5.  Após o plano (ou mais perguntas), PERGUNTE se ele aceita as missões, deseja ajustes, ou respondendo à sua última pergunta.
"""
# ... (resto da Célula 2.1) ...
print("Célula 2.1: Estado da Sessão e Constantes do Sistema (AJUSTADAS) DEFINIDOS.")

Célula 2.1: Estado da Sessão e Constantes do Sistema (AJUSTADAS) DEFINIDOS.


In [4]:
# Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADA)

def marcar_habitos_concluidos_ui(lista_nomes_habitos_concluidos_da_ui):
    global xp_total, gold_total, plano_de_habitos_gerado, nivel_usuario # Acessa as globais da Célula 2.1

    print(f"DEBUG - marcar_habitos_concluidos_ui: Recebido da UI - {lista_nomes_habitos_concluidos_da_ui}")

    # Primeiro, desmarca todos os hábitos que não estão na lista da UI (se foram deselecionados)
    for habito_dict in plano_de_habitos_gerado:
        nome_habito_no_plano = habito_dict.get("nome", "")
        # Lógica de desmarcar se o hábito estava concluído mas não está mais na lista de selecionados
        if habito_dict.get("concluido", False) and nome_habito_no_plano not in lista_nomes_habitos_concluidos_da_ui:
            habito_dict["concluido"] = False
            xp_total -= habito_dict["xp"]
            gold_total -= habito_dict["gold"]
            if xp_total < 0: xp_total = 0
            if gold_total < 0: gold_total = 0
            print(f"DEBUG: Hábito '{habito_dict['nome']}' DESMARCADO via UI. Pontos revertidos.")

    # Agora, marca os hábitos que estão na lista da UI e ainda não foram marcados como concluídos no backend
    for nome_habito_concluido_ui in lista_nomes_habitos_concluidos_da_ui:
        habito_alvo = None
        for habito_dict_interno in plano_de_habitos_gerado:
            if habito_dict_interno.get("nome") == nome_habito_concluido_ui:
                habito_alvo = habito_dict_interno
                break

        if habito_alvo and not habito_alvo.get("concluido", False): # Só concede se não estava concluído
            habito_alvo["concluido"] = True
            xp_total += habito_alvo["xp"]
            gold_total += habito_alvo["gold"]
            print(f"DEBUG: Hábito '{habito_alvo['nome']}' MARCADO como concluído via UI! +{habito_alvo['xp']} XP, +{habito_alvo['gold']} Gold.")

            xp_para_proximo_nivel = nivel_usuario * 100
            if xp_total >= xp_para_proximo_nivel:
                nivel_usuario += 1
                # xp_total -= xp_para_proximo_nivel # Opcional: resetar XP para o próximo nível
                print(f"DEBUG: LEVEL UP! Você alcançou o Nível {nivel_usuario}!")
        elif habito_alvo and habito_alvo.get("concluido", False):
             print(f"DEBUG: Hábito '{nome_habito_concluido_ui}' já estava marcado como concluído no backend.")
        elif not habito_alvo:
            print(f"DEBUG: Hábito com nome '{nome_habito_concluido_ui}' (da UI) não encontrado no plano interno.")

    return get_estado_jogo_atualizado()


def get_estado_jogo_atualizado():
    global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
    # Para o CheckboxGroup, precisamos de uma lista de nomes e uma lista dos que estão concluídos
    nomes_habitos = [h.get("nome", f"Missão ID {h.get('id')}") for h in plano_de_habitos_gerado]
    habitos_concluidos_nomes = [h.get("nome") for h in plano_de_habitos_gerado if h.get("concluido", False)]

    return {
        "xp": xp_total,
        "gold": gold_total,
        "nivel": nivel_usuario,
        "habitos_detalhados": plano_de_habitos_gerado, # Lista completa para referência interna
        "nomes_habitos_para_checkboxgroup": nomes_habitos,
        "habitos_concluidos_para_checkboxgroup": habitos_concluidos_nomes
    }

def tentar_parsear_plano_de_habitos_da_ia(texto_resposta_ia_str):
    global plano_de_habitos_gerado, id_habito_counter

    habitos_extraidos_lista = []
    if "### PERGAMINHO DAS MISSÕES INICIAIS ###" not in texto_resposta_ia_str:
        print("DEBUG - tentar_parsear: Marcador de início do plano não encontrado.")
        return False

    linhas = texto_resposta_ia_str.split('\n')
    missao_atual_nome = None
    detalhes_habito_atual = {}

    # Resetar o contador de ID se um novo plano está sendo parseado
    # para evitar IDs muito grandes se o usuário gerar múltiplos planos na mesma sessão
    # Ou, melhor ainda, resetar o plano_de_habitos_gerado antes de popular
    plano_de_habitos_gerado = [] # Limpa o plano anterior antes de parsear um novo
    id_habito_counter = 0 # Reseta o contador de ID


    for linha_str in linhas:
        linha_strip = linha_str.strip()

        if linha_strip.startswith("**Missão:"):
            if missao_atual_nome and 'meta' in detalhes_habito_atual:
                detalhes_habito_atual['id'] = id_habito_counter
                id_habito_counter += 1
                detalhes_habito_atual['concluido'] = False
                detalhes_habito_atual.setdefault('frequencia', 'Diário')
                detalhes_habito_atual.setdefault('xp', 20)
                detalhes_habito_atual.setdefault('gold', 2)
                habitos_extraidos_lista.append(detalhes_habito_atual)

            missao_atual_nome = linha_strip.replace("**Missão:", "").replace("**", "").strip()
            detalhes_habito_atual = {"nome": missao_atual_nome}

        elif missao_atual_nome and linha_strip.startswith("- Meta:"):
            detalhes_habito_atual['meta'] = linha_strip.replace("- Meta:", "").strip()
        elif missao_atual_nome and linha_strip.startswith("- Frequência:"):
            detalhes_habito_atual['frequencia'] = linha_strip.replace("- Frequência:", "").strip()
        elif missao_atual_nome and linha_strip.startswith("- Recompensa:"):
            recompensa_str = linha_strip.replace("- Recompensa:", "").strip()
            try:
                partes_recompensa = recompensa_str.lower().split(',')
                xp_str = partes_recompensa[0].replace("xp", "").strip()
                detalhes_habito_atual['xp'] = int(xp_str)
                if len(partes_recompensa) > 1:
                    gold_str = partes_recompensa[1].replace("gold", "").strip()
                    detalhes_habito_atual['gold'] = int(gold_str)
                else:
                    detalhes_habito_atual['gold'] = int(int(xp_str) / 10) if int(int(xp_str) / 10) > 0 else 1
            except ValueError:
                print(f"DEBUG - tentar_parsear: Erro ao parsear XP/Gold para '{missao_atual_nome}'. Usando defaults.")
                detalhes_habito_atual['xp'] = 20
                detalhes_habito_atual['gold'] = 2

    if missao_atual_nome and 'meta' in detalhes_habito_atual and not any(d.get('nome') == missao_atual_nome for d in habitos_extraidos_lista):
        detalhes_habito_atual['id'] = id_habito_counter
        id_habito_counter += 1
        detalhes_habito_atual['concluido'] = False
        detalhes_habito_atual.setdefault('frequencia', 'Diário')
        detalhes_habito_atual.setdefault('xp', 20)
        detalhes_habito_atual.setdefault('gold', 2)
        habitos_extraidos_lista.append(detalhes_habito_atual)

    if habitos_extraidos_lista:
        plano_de_habitos_gerado = habitos_extraidos_lista
        print(f"DEBUG - tentar_parsear: Plano de hábitos ATUALIZADO da IA: {plano_de_habitos_gerado}")
        return True
    else:
        print("DEBUG - tentar_parsear: Nenhum hábito concreto foi parseado da resposta da IA.")
        return False

def resetar_jogo_e_conversa():
    global conversa_com_ia_para_habitos, plano_de_habitos_gerado, xp_total, gold_total, nivel_usuario, id_habito_counter
    conversa_com_ia_para_habitos = []
    plano_de_habitos_gerado = []
    xp_total = 0
    gold_total = 0
    nivel_usuario = 1
    id_habito_counter = 0 # Importante resetar o contador de ID também
    print("DEBUG: Jogo e conversa resetados.")
    return get_estado_jogo_atualizado()

print("Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADAS) DEFINIDAS.")

Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADAS) DEFINIDAS.


In [36]:
# Célula 3: Definição e Lançamento da Interface Gradio

import gradio as gr
import html
import markdown2 # Mantemos o import

# --- Definição do Tema ---
custom_theme_habituai = gr.themes.Base(
    primary_hue=gr.themes.colors.green,
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
).set(
    body_background_fill="#1A1A1A", body_background_fill_dark="#1A1A1A",
    block_background_fill="#242424", block_border_width="0px", block_shadow="*shadow_drop_lg",
    button_primary_background_fill=gr.themes.colors.green.c600, button_primary_text_color="white",
)

# --- CSS (css_baloes_ajustados da resposta anterior) ---
# O CSS permanece o mesmo, pois ele estiliza as classes que estamos gerando no HTML.
css_baloes_ajustados = """
#output_conversa_html_habituai { /* MUDADO: ID para o novo componente gr.HTML */
    background-color: #202020; padding: 15px 10px;
    border-radius: 8px; overflow-y: auto; height: 450px;
    display: flex; flex-direction: column-reverse;
}
#output_conversa_html_habituai::before { content: ''; flex-grow: 1; }
.message-wrapper {
    display: flex; flex-direction: column; margin-bottom: 12px;
    max-width: 78%; min-width: 60px;
}
.sender-name { font-size: 0.75em; color: #A0A0A0; margin-bottom: 3px; }
.message-bubble {
    padding: 9px 13px; border-radius: 18px; line-height: 1.45;
    word-wrap: break-word; font-size: 0.92em; display: inline-block;
}
.user-wrapper { align-self: flex-end; margin-left: auto; }
.user-wrapper .sender-name { text-align: right; padding-right: 13px; }
.message-bubble.user-bubble {
    background-color: #FFFFFF !important; color: #1D1D1D !important;
    border-radius: 18px 5px 18px 18px; align-self: flex-end;
}
.model-wrapper { align-self: flex-start; margin-right: auto; }
.model-wrapper .sender-name { text-align: left; padding-left: 13px; }
.message-bubble.model-bubble {
    background-color: #25D366 !important; color: #FFFFFF !important;
    border-radius: 5px 18px 18px 18px; align-self: flex-start;
}
.message-bubble.model-bubble strong { font-weight: bold; }
.message-bubble.model-bubble em { font-style: italic; }
.message-bubble.model-bubble ul, .message-bubble.model-bubble ol { margin-left: 20px; padding-left: 5px; list-style-position: outside; margin-top: 0.5em; margin-bottom: 0.5em;}
.message-bubble.model-bubble li { margin-bottom: 4px; }
.message-bubble.model-bubble p { margin-top: 0; margin-bottom: 0.5em; }

#chat_input_bar_habituai { background-color: #242424; padding: 8px; border-top: 1px solid #303030;}
#input_usuario_habituai textarea { border-radius: 18px !important; background-color: #2C2C2C !important; color: #E0E0E0 !important; border: 1px solid #383838 !important;}
#btn_enviar_habituai { border-radius: 50% !important; min-width: 45px !important; background-color: var(--button-primary-background-fill) !important; color: var(--button-primary-text-color) !important; border: none !important; font-size: 1.5em !important; width: 45px !important; height: 45px !important; display:flex; align-items:center; justify-content:center;}
.status-section { padding: 10px; background-color: var(--block-background-fill); border-radius: 8px; margin-bottom:15px;}
.status-section .gr-markdown h3 { margin-top:0; color: var(--primary-hue-500) !important;}
.gr-checkboxgroup label span {color: #E0E0E0 !important}
.gr-checkboxgroup .gr-check-radio span {color: #C0C0C0 !important}
#detalhes_habitos_display_md ul li { background-color: #2C2C2C !important; border-radius: 4px !important; margin-bottom: 5px !important; padding: 8px !important; list-style-type: none; }
#detalhes_habitos_display_md ul li strong { color: #E0E0E0 !important; }
#detalhes_habitos_display_md ul li small { color: #AAAAAA !important; }
"""

with gr.Blocks(theme=custom_theme_habituai, title="HabituAI", css=css_baloes_ajustados) as demo_habituai:
    estado_conversa_hist = gr.State([])
    estado_jogo_ui = gr.State({
        "xp": 0, "gold": 0, "nivel": 1, "habitos_detalhados": [],
        "nomes_habitos_para_checkboxgroup": [], "habitos_concluidos_para_checkboxgroup": []
    })

    gr.Markdown(
        """<div style='text-align: center;'>
        <h1 style='color: var(--primary-hue-500); font-family: "Georgia", serif; text-shadow: 1px 1px #000;'>⚔️ HabituAI ⚔️</h1>
        <p style='color: #B0B0B0; font-size: 1.1em;'>Seu Guia Pessoal para Forjar Hábitos Lendários!</p>
        </div>"""
    )
    with gr.Row():
        with gr.Column(scale=7):
            # MUDANÇA AQUI: de gr.Markdown para gr.HTML
            output_conversa_ia = gr.HTML(elem_id="output_conversa_html_habituai", value="<p style='color:#A0A0A0; text-align:center; margin-top: 20px;'><em>HabituAI está preparando sua jornada...</em></p>")
            with gr.Row(elem_id="chat_input_bar_habituai"):
                input_usuario_chat = gr.Textbox(elem_id="input_usuario_habituai", placeholder="Sua resposta ao HabituAI...", lines=1, show_label=False, scale=10)
                btn_enviar_mensagem = gr.Button("➤", elem_id="btn_enviar_habituai", scale=1, min_width=45)
        with gr.Column(scale=3):
            # ... (Seu conteúdo de status e missões como antes) ...
            with gr.Group(elem_classes="status-section"):
                gr.Markdown("### 📜 Seu Pergaminho")
                nivel_display = gr.Markdown(f"**Nível:** 1")
                xp_display = gr.Markdown(f"**XP:** 0")
                gold_display = gr.Markdown(f"**Gold:** 0")
            with gr.Group(elem_classes="status-section"):
                gr.Markdown("### 🔥 Missões Ativas")
                checkbox_habitos_ui = gr.CheckboxGroup(label="Marque as missões concluídas:", choices=[], value=[], interactive=True)
                detalhes_habitos_display_md = gr.Markdown("Nenhuma missão ativa no momento.", elem_id="detalhes_habitos_display_md")


    # --- Funções de Callback da UI ---
    def formatar_conversa_para_html_com_baloes(historico_lista): # Nome da função como na sua última versão funcional
        # Esta função agora retorna HTML que será renderizado diretamente pelo gr.HTML
        chat_html = ""
        for item in historico_lista:
            is_user = item["role"] == "user"
            wrapper_class = "user-wrapper" if is_user else "model-wrapper"
            bubble_class_for_div = "user-bubble" if is_user else "model-bubble"
            sender_name_text = "Você" if is_user else "HabituAI"

            raw_content = item.get('content', '')

            processed_content = ""
            if is_user:
                processed_content = html.escape(raw_content).replace('\n', '<br />')
            else:
                try:
                    # Converte o Markdown da IA para HTML usando markdown2
                    # Assegure-se que a IA está sendo instruída a retornar Markdown válido
                    processed_content = markdown2.markdown(
                        raw_content,
                        extras=["fenced-code-blocks", "break-on-newline", "smarty-pants", "tables", "task_list", "cuddled-lists", "nofollow"]
                    )
                except Exception as md_e:
                    print(f"DEBUG UI - Erro markdown2: {md_e}. Usando fallback para: {raw_content[:50]}")
                    processed_content = html.escape(raw_content).replace('\n', '<br />')

            chat_html += f"""
            <div class='message-wrapper {wrapper_class}'>
                <div class='sender-name'>{sender_name_text}</div>
                <div class='message-bubble {bubble_class_for_div}'>
                    {processed_content}
                </div>
            </div>
            """
        return chat_html

    # --- Coloque suas funções de callback da UI COMPLETAS AQUI ---
    # (atualizar_display_habitos_e_status_jogo_ui, ui_interagir_com_ia,
    #  ui_iniciar_conversa, checkbox_mudou_ui_wrapper)
    # Certifique-se de que elas estão exatamente como na sua última versão funcional
    # e que chamam `formatar_conversa_para_html_com_baloes` corretamente.
    # Copie-as da sua última versão funcional. Eu vou colocar placeholders resumidos.

    def atualizar_display_habitos_e_status_jogo_ui(estado_jogo_atualizado_dict):
        # (SUA FUNÇÃO COMPLETA AQUI)
        choices = estado_jogo_atualizado_dict.get("nomes_habitos_para_checkboxgroup", [])
        value = estado_jogo_atualizado_dict.get("habitos_concluidos_para_checkboxgroup", [])
        detalhes_md_txt = "" # Esta será HTML agora, pois o output é gr.Markdown
        habitos_detalhados = estado_jogo_atualizado_dict.get("habitos_detalhados", [])
        if habitos_detalhados:
            detalhes_md_txt = "<ul id='detalhes_habitos_lista_ui'>" # ID para CSS
            for h in habitos_detalhados:
                s_emoji = "✅" if h.get("concluido") else "⏳"
                nome_h_html = f"<strong>{html.escape(h.get('nome', 'N/A'))}</strong>" # Usa strong para negrito
                meta_h_html = html.escape(h.get('meta', 'N/A'))
                freq_h_html = html.escape(h.get('frequencia', 'N/A'))
                xp_h = h.get('xp', 0)
                gold_h = h.get('gold', 0)
                detalhes_md_txt += (
                    f"<li>{s_emoji} {nome_h_html}<br>"
                    f"<small class='mission-details'>🎯 Meta: {meta_h_html} | 🗓️ Freq: {freq_h_html}<br>"
                    f"✨ Recompensa: {xp_h} XP, {gold_h} Gold</small></li>"
                )
            detalhes_md_txt += "</ul>"
        else:
            detalhes_md_txt = "<p style='color: #9E9E9E;'>Nenhuma missão ativa no momento.</p>"
        return (gr.update(choices=choices, value=value), detalhes_md_txt,
                f"**Nível:** {estado_jogo_atualizado_dict.get('nivel', 1)}",
                f"**XP:** {estado_jogo_atualizado_dict.get('xp', 0)}",
                f"**Gold:** {estado_jogo_atualizado_dict.get('gold', 0)}",
                estado_jogo_atualizado_dict)

    def ui_interagir_com_ia(mensagem_usuario_str, historico_gr_state_lista):
        # (SUA FUNÇÃO COMPLETA AQUI)
        global conversa_com_ia_para_habitos
        conversa_com_ia_para_habitos = list(historico_gr_state_lista)
        if mensagem_usuario_str and mensagem_usuario_str.strip():
            conversa_com_ia_para_habitos.append({"role": "user", "content": mensagem_usuario_str})
        instrucao_usada = INSTRUCAO_SISTEMA_BOAS_VINDAS
        mensagens_reais_usuario = [msg for msg in conversa_com_ia_para_habitos if msg["role"] == "user" and msg["content"] != "Começar jornada!"]
        if len(mensagens_reais_usuario) > 0:
            instrucao_usada = INSTRUCAO_SISTEMA_ANALISE_E_PLANOS
        resposta_ia_texto, historico_atualizado_completo = chamar_ia_para_habitos(conversa_com_ia_para_habitos, "", instrucao_usada)
        conversa_com_ia_para_habitos = historico_atualizado_completo
        chat_html = formatar_conversa_para_html_com_baloes(conversa_com_ia_para_habitos)
        if not (isinstance(resposta_ia_texto, str) and resposta_ia_texto.startswith("ErroAPI_")):
            if instrucao_usada == INSTRUCAO_SISTEMA_ANALISE_E_PLANOS or "PERGAMINHO DAS MISSÕES INICIAIS" in resposta_ia_texto:
                tentar_parsear_plano_de_habitos_da_ia(resposta_ia_texto)
        estado_jogo_dict = get_estado_jogo_atualizado()
        up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo = atualizar_display_habitos_e_status_jogo_ui(estado_jogo_dict)
        return (chat_html, conversa_com_ia_para_habitos, up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo)

    def ui_iniciar_conversa():
        # (SUA FUNÇÃO COMPLETA AQUI)
        global conversa_com_ia_para_habitos
        estado_resetado_dict = resetar_jogo_e_conversa()
        primeira_resposta_ia_texto, historico_inicial_com_ambas_msgs = chamar_ia_para_habitos([], "Começar jornada!", INSTRUCAO_SISTEMA_BOAS_VINDAS)
        conversa_com_ia_para_habitos = historico_inicial_com_ambas_msgs
        chat_html_inicial = formatar_conversa_para_html_com_baloes(conversa_com_ia_para_habitos)
        up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo = atualizar_display_habitos_e_status_jogo_ui(estado_resetado_dict)
        print(f"DEBUG UI: ui_iniciar_conversa - Chat HTML: '{chat_html_inicial[:150]}...' Histórico: {conversa_com_ia_para_habitos}")
        return (chat_html_inicial, conversa_com_ia_para_habitos, up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo)

    def checkbox_mudou_ui_wrapper(lista_habitos_selecionados_ui, estado_jogo_atual_gr_dict):
        # (SUA FUNÇÃO COMPLETA AQUI)
        global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
        xp_total = estado_jogo_atual_gr_dict["xp"]
        gold_total = estado_jogo_atual_gr_dict["gold"]
        nivel_usuario = estado_jogo_atual_gr_dict["nivel"]
        plano_de_habitos_gerado = list(estado_jogo_atual_gr_dict.get("habitos_detalhados", []))
        estado_jogo_apos_marcar = marcar_habitos_concluidos_ui(lista_habitos_selecionados_ui)
        return atualizar_display_habitos_e_status_jogo_ui(estado_jogo_apos_marcar)
    # --- FIM DAS FUNÇÕES DE CALLBACK ---

    # --- Conexões ---
    # (Mesmas conexões da sua versão anterior)
    btn_enviar_mensagem.click(
        fn=ui_interagir_com_ia,
        inputs=[input_usuario_chat, estado_conversa_hist],
        outputs=[output_conversa_ia, estado_conversa_hist, checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
    ).then(lambda: gr.update(value=""), outputs=input_usuario_chat)
    checkbox_habitos_ui.change(
        fn=checkbox_mudou_ui_wrapper,
        inputs=[checkbox_habitos_ui, estado_jogo_ui],
        outputs=[checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
    )
    demo_habituai.load(
        fn=ui_iniciar_conversa,
        inputs=None,
        outputs=[output_conversa_ia, estado_conversa_hist, checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
    )

    with gr.TabItem("🏰 Loja do Aventureiro", id="loja"):
        gr.Markdown("## 🛍️ Empório ...🚧 EM CONSTRUÇÃO 🚧")
    gr.Markdown("<hr><p style='text-align:center; font-size:0.9em; color:#757575;'>Uma criação da Imersão IA Alura com HabituAI</p>")

print("Iniciando a interface Gradio do HabituAI (Usando gr.HTML para chat)...")
demo_habituai.launch(debug=True, share=True)

Iniciando a interface Gradio do HabituAI (Usando gr.HTML para chat)...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://83728ebf21bf2bf3bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG Backend: Jogo e conversa resetados.
DEBUG Backend: get_estado_jogo_atualizado retornando: {'xp': 0, 'gold': 0, 'nivel': 1, 'habitos_detalhados': [], 'nomes_habitos_para_checkboxgroup': [], 'habitos_concluidos_para_checkboxgroup': []}
DEBUG - chamar_ia: Nova msg user: 'Começar jornada!' | Usando instrução: '
Você é 'HabituAI', seu Guia P...'
DEBUG - chamar_ia: Enviando para API (últimos 500 chars do prompt): ...User: Começar jornada!
DEBUG - chamar_ia: Resposta IA: 'Olá, Aventureiro! Sou HabituAI, seu guia nesta jornada de transformação de hábitos.  Para começarmos...'
DEBUG UI: ui_iniciar_conversa - Chat HTML: '
            <div class='message-wrapper user-wrapper'>
                <div class='sender-name'>Você</div>
                <div class='message-bubble...' Histórico: [{'role': 'user', 'content': 'Começar jornada!'}, {'role': 'model', 'content': 'Olá, Aventureiro! Sou HabituAI, seu guia nesta jornada de transformação de hábitos.  Para começarmos nossa aventura, me diga: qua

In [ ]:
# Célula 3: Definição e Lançamento da Interface Gradio (Login Corrigido + Display de Usuário/Nível)

import gradio as gr
import html
import markdown2

# --- Definição do Tema (como antes) ---
custom_theme_habituai = gr.themes.Base(
    primary_hue=gr.themes.colors.green,
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
).set(
    body_background_fill="#1A1A1A", body_background_fill_dark="#1A1A1A",
    block_background_fill="#242424", block_border_width="0px", block_shadow="*shadow_drop_lg",
    button_primary_background_fill=gr.themes.colors.green.c600, button_primary_text_color="white",
)

# --- CSS (css_login_e_chat da resposta anterior) ---
css_login_e_chat = """
/* ... (SEU CSS COMPLETO E CORRIGIDO DA RESPOSTA ANTERIOR PARA O CHAT VAI AQUI) ... */
#output_conversa_html_habituai { /* Certifique-se que este ID está no seu CSS se usar gr.HTML */
    background-color: #202020; padding: 15px 10px;
    border-radius: 8px; overflow-y: auto; height: 450px;
    display: flex; flex-direction: column-reverse;
}
#output_conversa_html_habituai::before { content: ''; flex-grow: 1; }
.message-wrapper { display: flex; flex-direction: column; margin-bottom: 12px; max-width: 78%; min-width: 60px; }
.sender-name { font-size: 0.75em; color: #A0A0A0; margin-bottom: 3px; }
.message-bubble { padding: 9px 13px; border-radius: 18px; line-height: 1.45; word-wrap: break-word; font-size: 0.92em; display: inline-block; }
.user-wrapper { align-self: flex-end; margin-left: auto; }
.user-wrapper .sender-name { text-align: right; padding-right: 13px; }
.message-bubble.user-bubble { background-color: #FFFFFF !important; color: #1D1D1D !important; border-radius: 18px 5px 18px 18px; align-self: flex-end; }
.model-wrapper { align-self: flex-start; margin-right: auto; }
.model-wrapper .sender-name { text-align: left; padding-left: 13px; }
.message-bubble.model-bubble { background-color: #25D366 !important; color: #FFFFFF !important; border-radius: 5px 18px 18px 18px; align-self: flex-start; }
.message-bubble.model-bubble strong { font-weight: bold; } .message-bubble.model-bubble em { font-style: italic; }
.message-bubble.model-bubble ul, .message-bubble.model-bubble ol { margin-left: 20px; padding-left: 5px; list-style-position: outside; margin-top: 0.5em; margin-bottom: 0.5em;}
.message-bubble.model-bubble li { margin-bottom: 4px; } .message-bubble.model-bubble p { margin-top: 0; margin-bottom: 0.5em; }
#chat_input_bar_habituai { background-color: #242424; padding: 8px; border-top: 1px solid #303030;}
#input_usuario_habituai textarea { border-radius: 18px !important; background-color: #2C2C2C !important; color: #E0E0E0 !important; border: 1px solid #383838 !important;}
#btn_enviar_habituai { border-radius: 50% !important; min-width: 45px !important; background-color: var(--button-primary-background-fill) !important; color: var(--button-primary-text-color) !important; border: none !important; font-size: 1.5em !important; width: 45px !important; height: 45px !important; display:flex; align-items:center; justify-content:center;}
.status-section { padding: 10px; background-color: var(--block-background-fill); border-radius: 8px; margin-bottom:15px;}
.status-section .gr-markdown h3 { margin-top:0; color: var(--primary-hue-500) !important;}
.gr-checkboxgroup label span {color: #E0E0E0 !important} .gr-checkboxgroup .gr-check-radio span {color: #C0C0C0 !important}
#detalhes_habitos_display_md ul li { background-color: #2C2C2C !important; border-radius: 4px !important; margin-bottom: 5px !important; padding: 8px !important; list-style-type: none; }
#detalhes_habitos_display_md ul li strong { color: #E0E0E0 !important; } #detalhes_habitos_display_md ul li small { color: #AAAAAA !important; }

/* Estilos para a tela de Login */
#login_screen_wrapper { display: flex; flex-direction: column; justify-content: center; align-items: center; min-height: 50vh; padding-top: 20px; }
#login_block { padding: 25px 30px; max-width: 380px; width: 90%; background-color: var(--block-background-fill); border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.2); }
#login_block .gr-button-primary { width: 100%; margin-top:20px !important; padding: 10px !important; font-size: 1.05em !important;}
#login_error_message { color: #FF6B6B; margin-top: 15px; text-align: center; font-size:0.9em; min-height: 20px;}
#login_title { text-align:center; color: var(--primary-hue-500); margin-bottom:25px !important; font-size:1.8em; font-weight:bold; }
#app_header_habituai { padding: 10px 20px; background-color: #202020; color: #E0E0E0; display: flex; justify-content: space-between; align-items: center; }
#user_info_display { font-size: 0.9em; text-align: right; }
"""

# --- CREDENCIAIS SIMULADAS ---
USUARIO_VALIDO = "aventureiro" # Sem espaços, minúsculas para simplificar
SENHA_VALIDA = "habito123"

with gr.Blocks(theme=custom_theme_habituai, title="HabituAI", css=css_login_e_chat) as demo_habituai_com_login:

    # Estados Globais da UI
    usuario_logado_estado = gr.State(False)
    nome_usuario_logado_estado = gr.State("") # Novo estado para o nome do usuário

    # --- TELA DE LOGIN (visível por padrão) ---
    with gr.Column(visible=True, elem_id="login_screen_wrapper") as tela_login_wrapper:
        with gr.Group(elem_id="login_block"):
            gr.Markdown("⚔️ HabituAI ⚔️", elem_id="login_title")
            login_usuario_input = gr.Textbox(label="Nome do Aventureiro", placeholder="Digite: aventureiro")
            login_senha_input = gr.Textbox(label="Senha Secreta", placeholder="Digite: habito123", type="password")
            btn_entrar = gr.Button("Entrar na Guilda", variant="primary")
            login_status_msg = gr.Markdown(elem_id="login_error_message")

    # --- TELA PRINCIPAL DO HABITFAI (Inicialmente oculta) ---
    with gr.Column(visible=False) as tela_principal_habituai:
        # Header com nome do usuário e nível
        with gr.Row(elem_id="app_header_habituai"):
            app_title_main = gr.Markdown("<h1 style='margin:0; font-size: 1.6em; color: var(--primary-hue-500);'>⚔️ HabituAI ⚔️</h1>")
            user_info_display = gr.Markdown(elem_id="user_info_display") # Para "Usuário: [Nome], Nível: [Nível]"

        # Estados do Jogo e Chat (específicos para quando o usuário está logado)
        estado_conversa_hist = gr.State([])
        estado_jogo_ui = gr.State({
            "xp": 0, "gold": 0, "nivel": 1, "habitos_detalhados": [],
            "nomes_habitos_para_checkboxgroup": [], "habitos_concluidos_para_checkboxgroup": []
        })

        # Layout da interface principal do HabituAI
        with gr.Tabs() as tabs_main:
            with gr.TabItem("💬 Chat com HabituAI / 🔥 Missões", id="chat_missoes"):
                with gr.Row():
                    with gr.Column(scale=7):
                        output_conversa_ia = gr.HTML(elem_id="output_conversa_html_habituai", value="<p style='color:#A0A0A0; text-align:center; margin-top: 20px;'><em>HabituAI aguarda seu comando...</em></p>")
                        with gr.Row(elem_id="chat_input_bar_habituai"):
                            input_usuario_chat = gr.Textbox(elem_id="input_usuario_habituai", placeholder="Sua resposta ao HabituAI...", lines=1, show_label=False, scale=10)
                            btn_enviar_mensagem = gr.Button("➤", elem_id="btn_enviar_habituai", scale=1, min_width=45)
                    with gr.Column(scale=3):
                        with gr.Group(elem_classes="status-section"):
                            gr.Markdown("### 📜 Seu Pergaminho")
                            nivel_display_main = gr.Markdown(f"**Nível:** 1") # Renomeado para evitar conflito com o da tela de login (se houver)
                            xp_display = gr.Markdown(f"**XP:** 0")
                            gold_display = gr.Markdown(f"**Gold:** 0")
                        with gr.Group(elem_classes="status-section"):
                            gr.Markdown("### 🔥 Missões Ativas")
                            checkbox_habitos_ui = gr.CheckboxGroup(label="Marque as missões concluídas:", choices=[], value=[], interactive=True)
                            detalhes_habitos_display_md = gr.Markdown("Nenhuma missão ativa no momento.", elem_id="detalhes_habitos_display_md")

            with gr.TabItem("🏰 Loja do Aventureiro", id="loja"):
                 gr.Markdown("## 🛍️ Empório ...🚧 EM CONSTRUÇÃO 🚧")

        gr.Markdown("<hr><p style='text-align:center; font-size:0.9em; color:#757575;'>Uma criação da Imersão IA Alura com HabituAI</p>")

        # --- Funções de Callback do HabituAI (COPIE SUAS FUNÇÕES COMPLETAS AQUI) ---
        # (formatar_conversa_para_html_com_baloes, atualizar_display_habitos_e_status_jogo_ui,
        #  ui_interagir_com_ia, ui_iniciar_conversa_habituai, checkbox_mudou_ui_wrapper)
        # Copie-as da sua Célula 3 anterior que estava funcional para o chat.
        # Eu vou incluir placeholders para a estrutura.

        def formatar_conversa_para_html_com_baloes(historico_lista):
            # (Sua função completa)
            chat_html = ""
            for item in historico_lista:
                is_user = item["role"] == "user"; wrapper_class = "user-wrapper" if is_user else "model-wrapper"
                bubble_class_for_div = "user-bubble" if is_user else "model-bubble"
                sender_name_text = "Você" if is_user else "HabituAI"; raw_content = item.get('content', '')
                processed_content = "";
                if is_user: processed_content = html.escape(raw_content).replace('\n', '<br />')
                else:
                    try: processed_content = markdown2.markdown(raw_content, extras=["fenced-code-blocks", "break-on-newline", "smarty-pants", "tables", "task_list", "cuddled-lists", "nofollow"])
                    except Exception: processed_content = html.escape(raw_content).replace('\n', '<br />')
                chat_html += f"<div class='message-wrapper {wrapper_class}'><div class='sender-name'>{sender_name_text}</div><div class='message-bubble {bubble_class_for_div}'>{processed_content}</div></div>"
            return chat_html

        def atualizar_display_habitos_e_status_jogo_ui(estado_jogo_atualizado_dict):
            # (Sua função completa)
            choices = estado_jogo_atualizado_dict.get("nomes_habitos_para_checkboxgroup", []); value = estado_jogo_atualizado_dict.get("habitos_concluidos_para_checkboxgroup", [])
            detalhes_md_txt = ""; habitos_detalhados = estado_jogo_atualizado_dict.get("habitos_detalhados", [])
            if habitos_detalhados:
                detalhes_md_txt = "<ul id='detalhes_habitos_lista_ui'>"
                for h in habitos_detalhados:
                    s_emoji = "✅" if h.get("concluido") else "⏳"; nome_h_html = f"<strong>{html.escape(h.get('nome', 'N/A'))}</strong>"
                    meta_h_html = html.escape(h.get('meta', 'N/A')); freq_h_html = html.escape(h.get('frequencia', 'N/A'))
                    xp_h = h.get('xp', 0); gold_h = h.get('gold', 0)
                    detalhes_md_txt += (f"<li>{s_emoji} {nome_h_html}<br><small class='mission-details'>🎯 Meta: {meta_h_html} | 🗓️ Freq: {freq_h_html}<br>✨ Recompensa: {xp_h} XP, {gold_h} Gold</small></li>")
                detalhes_md_txt += "</ul>"
            else: detalhes_md_txt = "<p style='color: #9E9E9E;'>Nenhuma missão ativa.</p>"
            return (gr.update(choices=choices, value=value), detalhes_md_txt,
                    f"**Nível:** {estado_jogo_atualizado_dict.get('nivel', 1)}",
                    f"**XP:** {estado_jogo_atualizado_dict.get('xp', 0)}",
                    f"**Gold:** {estado_jogo_atualizado_dict.get('gold', 0)}",
                    estado_jogo_atualizado_dict)

        def ui_interagir_com_ia(mensagem_usuario_str, historico_gr_state_lista, estado_jogo_atual_gr_dict):
            # (Sua função completa)
            global conversa_com_ia_para_habitos, xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
            # Sincroniza globais com o estado da UI
            conversa_com_ia_para_habitos = list(historico_gr_state_lista)
            xp_total = estado_jogo_atual_gr_dict.get("xp",0); gold_total = estado_jogo_atual_gr_dict.get("gold",0)
            nivel_usuario = estado_jogo_atual_gr_dict.get("nivel",1); plano_de_habitos_gerado = list(estado_jogo_atual_gr_dict.get("habitos_detalhados",[]))

            if mensagem_usuario_str and mensagem_usuario_str.strip():
                conversa_com_ia_para_habitos.append({"role": "user", "content": mensagem_usuario_str})
            instrucao_usada = INSTRUCAO_SISTEMA_BOAS_VINDAS
            mensagens_reais_usuario = [msg for msg in conversa_com_ia_para_habitos if msg["role"] == "user" and msg["content"] != "Começar jornada!"]
            if len(mensagens_reais_usuario) > 0: instrucao_usada = INSTRUCAO_SISTEMA_ANALISE_E_PLANOS

            resposta_ia_texto, historico_api = chamar_ia_para_habitos(conversa_com_ia_para_habitos, "", instrucao_usada)
            conversa_com_ia_para_habitos = historico_api

            chat_html = formatar_conversa_para_html_com_baloes(conversa_com_ia_para_habitos)
            if not (isinstance(resposta_ia_texto, str) and resposta_ia_texto.startswith("ErroAPI_")):
                if instrucao_usada == INSTRUCAO_SISTEMA_ANALISE_E_PLANOS or "PERGAMINHO DAS MISSÕES INICIAIS" in resposta_ia_texto:
                    tentar_parsear_plano_de_habitos_da_ia(resposta_ia_texto)

            estado_jogo_dict_atualizado_backend = get_estado_jogo_atualizado() # Pega das globais atualizadas
            up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_dict_retorno = atualizar_display_habitos_e_status_jogo_ui(estado_jogo_dict_atualizado_backend)

            # Atualiza o display de usuário no header
            nome_usuario_atual = nome_usuario_logado_estado.value # Pega do gr.State
            user_info_header_update = f"Aventureiro: **{html.escape(nome_usuario_atual)}** | Nível: **{estado_jogo_dict_atualizado_backend.get('nivel',1)}**"

            return (chat_html, conversa_com_ia_para_habitos, up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_dict_retorno, user_info_header_update)


        def ui_iniciar_conversa_habituai(nome_usuario_str): # Recebe o nome do usuário logado
            # (Sua função ui_iniciar_conversa, adaptada)
            global conversa_com_ia_para_habitos
            estado_resetado_dict = resetar_jogo_e_conversa() # Globais são resetadas
            primeira_resposta_ia_texto, historico_inicial = chamar_ia_para_habitos([], f"Começar jornada como {nome_usuario_str}!", INSTRUCAO_SISTEMA_BOAS_VINDAS)
            conversa_com_ia_para_habitos = historico_inicial
            chat_html_inicial = formatar_conversa_para_html_com_baloes(conversa_com_ia_para_habitos)
            up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_dict_retorno = atualizar_display_habitos_e_status_jogo_ui(estado_resetado_dict)

            user_info_header_text = f"Aventureiro: **{html.escape(nome_usuario_str)}** | Nível: **{estado_resetado_dict.get('nivel',1)}**"

            print(f"DEBUG UI: ui_iniciar_conversa_habituai - Chat HTML: '{chat_html_inicial[:150]}...'")
            return (chat_html_inicial, conversa_com_ia_para_habitos, up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_dict_retorno, user_info_header_text)

        def checkbox_mudou_ui_wrapper(lista_habitos_selecionados_ui, estado_jogo_atual_gr_dict, nome_usuario_str_estado):
            # (Sua função completa)
            global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
            xp_total = estado_jogo_atual_gr_dict.get("xp",0); gold_total = estado_jogo_atual_gr_dict.get("gold",0)
            nivel_usuario = estado_jogo_atual_gr_dict.get("nivel",1); plano_de_habitos_gerado = list(estado_jogo_atual_gr_dict.get("habitos_detalhados",[]))
            estado_jogo_apos_marcar = marcar_habitos_concluidos_ui(lista_habitos_selecionados_ui) # Atualiza globais

            # Atualiza os displays de status
            up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_retorno = atualizar_display_habitos_e_status_jogo_ui(estado_jogo_apos_marcar)

            # Atualiza o display de usuário no header
            user_info_header_update = f"Aventureiro: **{html.escape(nome_usuario_str_estado)}** | Nível: **{estado_jogo_apos_marcar.get('nivel',1)}**"

            return (up_check, up_det, up_niv, up_xp, up_gold, up_est_jogo_retorno, user_info_header_update)

        # --- Conexões da UI do HabituAI ---
        # Nota: `outputs` do btn_enviar_mensagem e checkbox_mudou_ui agora incluem user_info_display
        btn_enviar_mensagem.click(
            fn=ui_interagir_com_ia,
            inputs=[input_usuario_chat, estado_conversa_hist, estado_jogo_ui],
            outputs=[output_conversa_ia, estado_conversa_hist, checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display_main, xp_display, gold_display, estado_jogo_ui, user_info_display]
        ).then(lambda: gr.update(value=""), outputs=input_usuario_chat)

        checkbox_habitos_ui.change(
            fn=checkbox_mudou_ui_wrapper,
            inputs=[checkbox_habitos_ui, estado_jogo_ui, nome_usuario_logado_estado], # Adicionado nome_usuario_logado_estado
            outputs=[checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display_main, xp_display, gold_display, estado_jogo_ui, user_info_display]
        )

    # --- Função de Callback para o Login ---
    def tentar_login(usuario_input_str, senha_input_str):
        print(f"DEBUG Login: Tentando login com Usuário='{usuario_input_str}', Senha='{'*' * len(senha_input_str)}'")
        # Remove espaços extras e converte para minúsculas para comparação mais robusta
        usuario_tratado = usuario_input_str.strip().lower()
        senha_tratada = senha_input_str.strip()

        if usuario_tratado == USUARIO_VALIDO.lower() and senha_tratada == SENHA_VALIDA:
            print(f"DEBUG Login: Login BEM-SUCEDIDO para {usuario_tratado}")

            # Chama a função para iniciar a conversa do HabituAI e obter seus outputs iniciais
            # Passa o nome de usuário para ser usado na inicialização
            habituai_initial_outputs = ui_iniciar_conversa_habituai(usuario_tratado)

            return {
                tela_login_wrapper: gr.update(visible=False),
                tela_principal_habituai: gr.update(visible=True),
                login_status_msg: gr.update(value="Login bem-sucedido! Carregando sua aventura...", visible=True),
                usuario_logado_estado: True,
                nome_usuario_logado_estado: usuario_tratado, # Armazena o nome do usuário logado
                # Mapeia os retornos de ui_iniciar_conversa_habituai para os componentes corretos
                output_conversa_ia: habituai_initial_outputs[0],
                estado_conversa_hist: habituai_initial_outputs[1],
                checkbox_habitos_ui: habituai_initial_outputs[2],
                detalhes_habitos_display_md: habituai_initial_outputs[3],
                nivel_display_main: habituai_initial_outputs[4], # Atualiza o nivel_display_main na tela principal
                xp_display: habituai_initial_outputs[5],
                gold_display: habituai_initial_outputs[6],
                estado_jogo_ui: habituai_initial_outputs[7],
                user_info_display: habituai_initial_outputs[8] # Atualiza o user_info_display no header
            }
        else:
            print(f"DEBUG Login: Login FALHOU para {usuario_tratado}")
            return {
                tela_login_wrapper: gr.update(visible=True),
                tela_principal_habituai: gr.update(visible=False),
                login_status_msg: gr.update(value="Usuário ou senha inválidos. Tente 'aventureiro' / 'habito123'.", visible=True),
                usuario_logado_estado: False,
                nome_usuario_logado_estado: "",
                # Retorna updates vazios ou padrão para os componentes da tela principal
                output_conversa_ia: gr.update(value=""), estado_conversa_hist: gr.update(value=[]),
                checkbox_habitos_ui: gr.update(choices=[], value=[]), detalhes_habitos_display_md: gr.update(value=""),
                nivel_display_main: gr.update(value="**Nível:** 1"), xp_display: gr.update(value="**XP:** 0"),
                gold_display: gr.update(value="**Gold:** 0"), estado_jogo_ui: gr.update(value={}),
                user_info_display: gr.update(value="")
            }

    # Conexão do botão de login
    btn_entrar.click(
        fn=tentar_login,
        inputs=[login_usuario_input, login_senha_input],
        outputs=[ # A ordem deve ser exata
            tela_login_wrapper, tela_principal_habituai, login_status_msg, usuario_logado_estado, nome_usuario_logado_estado,
            output_conversa_ia, estado_conversa_hist,
            checkbox_habitos_ui, detalhes_habitos_display_md,
            nivel_display_main, xp_display, gold_display, estado_jogo_ui,
            user_info_display # Output para o novo display de info do usuário
        ]
    )

# Lançamento da interface
print("Iniciando a interface Gradio do HabituAI com Login e Display de Usuário...")
demo_habituai_com_login.launch(debug=True, share=True)

Iniciando a interface Gradio do HabituAI com Login e Display de Usuário...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8bca59bf22ccfb1a55.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG Login: Tentando login com Usuário='gerso', Senha='*********'
DEBUG Login: Login FALHOU para gerso
DEBUG Login: Tentando login com Usuário='aventureiro', Senha='*********'
DEBUG Login: Login BEM-SUCEDIDO para aventureiro
DEBUG Backend: Jogo e conversa resetados.
DEBUG Backend: get_estado_jogo_atualizado retornando: {'xp': 0, 'gold': 0, 'nivel': 1, 'habitos_detalhados': [], 'nomes_habitos_para_checkboxgroup': [], 'habitos_concluidos_para_checkboxgroup': []}
DEBUG - chamar_ia: Nova msg user: 'Começar jornada como aventureiro!' | Usando instrução: '
Você é 'HabituAI', seu Guia P...'
DEBUG - chamar_ia: Enviando para API (últimos 500 chars do prompt): ...User: Começar jornada como aventureiro!
DEBUG - chamar_ia: Resposta IA: 'Olá, nobre Aventureiro!  Bem-vindo à sua jornada de transformação com HabituAI!  Prepare-se para for...'
DEBUG UI: ui_iniciar_conversa_habituai - Chat HTML: '<div class='message-wrapper user-wrapper'><div class='sender-name'>Você</div><div class='message-bubble us